# **Template for Torch-Pruning**

This template is just built for your convinience.

You are not required to follow the steps and method given below.

In [1]:
!pip install --upgrade torch_pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.3 MB/s eta 0:00:00


In [2]:
import torch
import torchvision
from torchvision.models import mobilenet_v2
import torch_pruning as tp

/home/leo/miniconda3/envs/Leo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##A Minimal Example ##  
In this section, you will perform channel pruning using the library [Torch-Pruning](https://github.com/VainF/Torch-Pruning).  

The puuner in Torch-Pruning has three main functions: sparse training (optional), importance estimation, and parameter removal.  
Torch-pruning offers two core features to support this process:

tp.importance(): This criteria is utilized to measure the importance of weights.  

tp.pruner(): This is a pruner used for the actual pruning of the parameters.  

For detailed information on this process, please refer to this [tutorial](https://github.com/VainF/Torch-Pruning/wiki/4.-High%E2%80%90level-Pruners/). Additionally, a more practical example is available in [here](https://github.com/VainF/Torch-Pruning/blob/master/benchmarks/main.py).

### 1. Load model


In [3]:
model = torch.load('./mobilenetv2_0.963.pth', map_location="cpu")
device = torch.device("cuda:0")
model = model.to(device)

### 2. Prepare a pruner
By default, Torch-Pruning will automatically prune the last non-singleton dim of these parameters. If you want to customize this behaviour, please provide an `unwrapped_parameters` list as the following example.

In [4]:
# Importance criterion
imp = tp.importance.GroupNormImportance(p=2, normalizer="max") # or GroupTaylorImportance(), GroupHessianImportance(), etc.

# Initialize a pruner with the model and the importance criterion
example_inputs = torch.randn(1, 3, 224, 224).to(device)

ignored_layers = []
for m in model.modules():
  if isinstance(m, torch.nn.Linear) and m.out_features == 10: # ignore the classifier
    ignored_layers.append(m)

pruner = tp.pruner.GroupNormPruner ( # you can choose any pruner you like.
    model,
    example_inputs,
    importance        = imp,
    pruning_ratio     = 0.75,
    max_pruning_ratio = 0.9,
    iterative_steps   = 200,
    ignored_layers    = ignored_layers,
    global_pruning    = True
)

### 3. Prune the model

In [3]:
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
from torchvision.models.mobilenetv2 import MobileNetV2

from tqdm import tqdm


transforms = {
    "train": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "test": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )

# You can apply your own batch_size
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=64,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def finetune(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR | None = None,
  callbacks = None,
  device: torch.device = torch.device("cpu"),
  l1_coeff: float | None = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.to(device)
    targets = targets.to(device)

    # Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Forward inference
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # L1 reg
    if l1_coeff:
      l1_loss = 0.
      for p in model.parameters():
        l1_loss += p.abs().sum()
      
      loss = loss + l1_coeff * l1_loss

    # Backward propagation
    loss.backward()

    # Update optimizer and LR scheduler
    optimizer.step()
    if scheduler: scheduler.step()

    if callbacks is not None:
        for callback in callbacks:
            callback()


@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  verbose=True,
  device: torch.device = torch.device("cpu")
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False,
                              disable=not verbose):
    # Move the data from CPU to GPU
    inputs = inputs.to(device)
    targets = targets.to(device)

    # Inference
    outputs = model(inputs)

    # Convert logits to class indices
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

In [7]:
# Parameters:
epochs_per_prune = 1
finetune_lr      = 0.01
target_MFLOPs    = 45.

# Model size before pruning
base_macs, base_nparams = tp.utils.count_ops_and_params(model, torch.randn(1, 3, 224, 224, device=device))
base_MFLOPs = base_macs / 1e6
print(f"[Original MFLOPs = {base_MFLOPs}]")

for i in range(1, pruner.iterative_steps + 1):
  
  model.eval()

  if isinstance(imp, tp.importance.GroupTaylorImportance):
    # Taylor expansion requires gradients for importance estimation
    loss = model(example_inputs).sum() # A dummy loss, please replace this line with your loss function and data!
    loss.backward() # before pruner.step()

  # prune
  pruner.step()

  # Parameter & MACs Counter
  pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, torch.randn(1, 3, 224, 224, device=device))
  MFLOPs = pruned_macs / 1e6
  print("\n" + "-" * 50 + f"Iter {i}" + "-" * 50)
  print(f"MFLOPs: {MFLOPs}")

  # Finetune the pruned model here
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=finetune_lr, momentum=0.9, weight_decay=1e-4)
  print(f"\nFinetuning ... ")
  finetune(model, dataloader['train'], criterion, optimizer, device=device)
  acc = evaluate(model, dataloader['test'], device=device)
  print(f"Accuracy: {acc}")

  # Stopping:
  if MFLOPs <= target_MFLOPs:
    break
  

[Original MFLOPs = 318.969098]

--------------------------------------------------Iter 1--------------------------------------------------
MFLOPs: 308.050134

Finetuning ... 


Accuracy: 95.1221923828125

--------------------------------------------------Iter 2--------------------------------------------------
MFLOPs: 299.791233

Finetuning ... 


Accuracy: 95.48277282714844

--------------------------------------------------Iter 3--------------------------------------------------
MFLOPs: 295.74839

Finetuning ... 


Accuracy: 95.58293151855469

--------------------------------------------------Iter 4--------------------------------------------------
MFLOPs: 294.269815

Finetuning ... 


Accuracy: 95.72315979003906

--------------------------------------------------Iter 5--------------------------------------------------
MFLOPs: 292.576571

Finetuning ... 


...

Accuracy: 89.17267608642578

--------------------------------------------------Iter 193--------------------------------------------------
MFLOPs: 47.821191

Finetuning ... 


Accuracy: 87.61017608642578

--------------------------------------------------Iter 194--------------------------------------------------
MFLOPs: 47.000078

Finetuning ... 


Accuracy: 89.0224380493164

--------------------------------------------------Iter 195--------------------------------------------------
MFLOPs: 46.032052

Finetuning ... 


Accuracy: 90.224365234375

--------------------------------------------------Iter 196--------------------------------------------------
MFLOPs: 44.467766

Finetuning ... 


Accuracy: 88.79206848144531


In [9]:
final_fintune_epochs = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, final_fintune_epochs)

best_acc = 0.
for e in range(1, final_fintune_epochs + 1):
    print(f"Epoch {e}:")
    finetune(model, dataloader['train'], criterion, optimizer, device=device)
    acc = evaluate(model, dataloader['test'], device=device)
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), f"./save/best_DepGraph_model_{best_acc :.4f}.pth")
    
    print(f"Accuracy = {acc} / Best = {best_acc}")

Epoch 1:


Accuracy = 92.46794891357422 / Best = 92.46794891357422
Epoch 2:


Accuracy = 92.54808044433594 / Best = 92.54808044433594
Epoch 3:


Accuracy = 92.8084945678711 / Best = 92.8084945678711
Epoch 4:


Accuracy = 92.46794891357422 / Best = 92.8084945678711
Epoch 5:


...

Accuracy = 93.08894348144531 / Best = 93.24919891357422
Epoch 97:


Accuracy = 93.11898803710938 / Best = 93.24919891357422
Epoch 98:


Accuracy = 92.98878479003906 / Best = 93.24919891357422
Epoch 99:


Accuracy = 93.14904022216797 / Best = 93.24919891357422
Epoch 100:


Accuracy = 93.1290054321289 / Best = 93.24919891357422


In [16]:
device = torch.device("cuda:0")

# Load last best model:
last_best_model_file = "save/best_DepGraph_2_model_93.1991.pth"
last_best_model = torch.load(last_best_model_file, map_location="cpu")
last_best_model.to(device)
last_best_model.eval()

# Importance criterion
imp = tp.importance.GroupNormImportance(p=2, normalizer="max") # or GroupTaylorImportance(), GroupHessianImportance(), etc.

# Initialize a pruner with the model and the importance criterion
example_inputs = torch.randn(1, 3, 224, 224).to(device)

ignored_layers = []
for m in last_best_model.modules():
  if isinstance(m, torch.nn.Linear) and m.out_features == 10:
    ignored_layers.append(m)

pruner = tp.pruner.GroupNormPruner(
    last_best_model,
    example_inputs,
    importance        = imp,
    pruning_ratio     = 0.2,
    iterative_steps   = 10,
    ignored_layers    = ignored_layers,
    global_pruning    = True
)

In [17]:
# Parameters:
epochs_per_prune = 1
finetune_lr      = 0.01
target_MFLOPs    = 27.

# Model size before pruning
base_macs, base_nparams = tp.utils.count_ops_and_params(last_best_model, torch.randn(1, 3, 224, 224, device=device))
base_MFLOPs = base_macs / 1e6
print(f"[Original MFLOPs = {base_MFLOPs}]")

for i in range(1, pruner.iterative_steps + 1):
  
  last_best_model.eval()

  if isinstance(imp, tp.importance.GroupTaylorImportance):
    # Taylor expansion requires gradients for importance estimation
    loss = last_best_model(example_inputs).sum() # A dummy loss, please replace this line with your loss function and data!
    loss.backward() # before pruner.step()

  # prune
  pruner.step()

  # Parameter & MACs Counter
  pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(last_best_model, torch.randn(1, 3, 224, 224, device=device))
  MFLOPs = pruned_macs / 1e6
  print("\n" + "-" * 50 + f"Iter {i}" + "-" * 50)
  print(f"MFLOPs: {MFLOPs}")

  # Finetune the pruned model here
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(last_best_model.parameters(), lr=finetune_lr, momentum=0.9, weight_decay=1e-5)
  print(f"\nFinetuning ... ")
  finetune(last_best_model, dataloader['train'], criterion, optimizer, device=device)
  acc = evaluate(last_best_model, dataloader['test'], device=device)
  print(f"Accuracy: {acc}")

  # Save ckpt
  torch.save(last_best_model, f"save/Prune{i}_{acc}.pth")

  # Stopping
  if MFLOPs <= target_MFLOPs:
    break

[Original MFLOPs = 32.86549]

--------------------------------------------------Iter 1--------------------------------------------------
MFLOPs: 31.66037

Finetuning ... 


Accuracy: 92.96875

--------------------------------------------------Iter 2--------------------------------------------------
MFLOPs: 31.461372

Finetuning ... 


Accuracy: 92.44792175292969

--------------------------------------------------Iter 3--------------------------------------------------
MFLOPs: 31.214874

Finetuning ... 


Accuracy: 92.62820434570312

--------------------------------------------------Iter 4--------------------------------------------------
MFLOPs: 30.456338

Finetuning ... 


Accuracy: 92.11738586425781

--------------------------------------------------Iter 5--------------------------------------------------
MFLOPs: 29.62826

Finetuning ... 


...

Accuracy: 88.57171630859375

--------------------------------------------------Iter 7--------------------------------------------------
MFLOPs: 28.307119

Finetuning ... 


Accuracy: 90.93550109863281

--------------------------------------------------Iter 8--------------------------------------------------
MFLOPs: 27.93037

Finetuning ... 


Accuracy: 89.92387390136719

--------------------------------------------------Iter 9--------------------------------------------------
MFLOPs: 27.47286

Finetuning ... 


Accuracy: 89.40304565429688

--------------------------------------------------Iter 10--------------------------------------------------
MFLOPs: 26.326102

Finetuning ... 


Accuracy: 89.43309020996094


In [18]:
final_fintune_epochs = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(last_best_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, final_fintune_epochs)

best_acc = 0.
for e in range(1, final_fintune_epochs + 1):
    print(f"Epoch {e}:")
    finetune(last_best_model, dataloader['train'], criterion, optimizer, device=device)
    acc = evaluate(last_best_model, dataloader['test'], device=device)
    if acc > best_acc:
        best_acc = acc
        torch.save(last_best_model, f"./save/best_next_DepGraph_model_{best_acc :.4f}.pth")
    
    print(f"Accuracy = {acc} / Best = {best_acc}")

Epoch 1:


Accuracy = 88.98237609863281 / Best = 88.98237609863281
Epoch 2:


Accuracy = 89.46314239501953 / Best = 89.46314239501953
Epoch 3:


Accuracy = 89.20272827148438 / Best = 89.46314239501953
Epoch 4:


Accuracy = 89.11257934570312 / Best = 89.46314239501953
Epoch 5:


...

Accuracy = 92.24759674072266 / Best = 92.24759674072266
Epoch 97:


Accuracy = 92.1875 / Best = 92.24759674072266
Epoch 98:


Accuracy = 92.24759674072266 / Best = 92.24759674072266
Epoch 99:


Accuracy = 92.24759674072266 / Best = 92.24759674072266
Epoch 100:


Accuracy = 92.31771087646484 / Best = 92.31771087646484
